In [ ]:
%reset

In [ ]:
from os import listdir
from os.path import exists
from importlib import reload
import numpy as np
import pandas as pd
import pyxdf
import mne
from utils import *
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.model_selection import train_test_split, cross_val_score, LeaveOneOut
import time
import datetime
from datetime import datetime, timezone
import pickle
import plotly.express as px
from scipy import stats

print('Imports done...')

In [ ]:
#data_path = 'C:/Users/tumfart/Code/github/master-thesis/data/'
data_path = 'C:/Users/peter/Google Drive/measurements/eeg/'
subjects = ['A01', 'A02', 'A03', 'A04', 'A05', 'A06', 'A07' , 'A08', 'A09', 'A10']
# = 'A03'
paradigm = 'paradigm' # 'eye', 'paradigm'
plot = False
mne.set_log_level('WARNING')
# Create path list for each subject:
paths = [str(data_path + subject + '/' + paradigm) for subject in subjects]

In [ ]:
# Read all .npy coefficients:
start = time.time()
A = []
for subject, path in zip(subjects, paths):
    file_names = [f for f in listdir(path) if 'regr_coeff.npy' in f]

    # Load file
    file_name = file_names[0]
    file = path + '/' + file_name
    A.append(np.load(file))

n_chan, n_coeff, n_ts = A[0].shape

In [ ]:
# Calculate mean for each channel and for each timestamp:
n_chan, n_coeff, n_ts = A[0].shape
means = np.empty((n_chan, n_coeff, n_ts))
uppers = np.empty((n_chan, n_coeff, n_ts))
lowers = np.empty((n_chan, n_coeff, n_ts))
means[:], uppers[:], lowers[:] = np.nan, np.nan, np.nan

stat = np.empty((n_chan, n_ts))
p_val = np.empty((n_chan, n_ts))
stat[:], p_val[:] = np.nan, np.nan

confidence = 0.95
start = time.time()
for ts in range(n_ts):
    for chan in range(n_chan):
        print(f'{ts+1}/{n_ts}, {chan+1}/{n_chan}', end='\r')
        for coeff in range(n_coeff):
            a = [A[i][chan, coeff, ts] for i in range(len(subjects))] # Get list of all subject for a specific channel, coefficient and timestamp

            # Calculate mean:
            # Bootstrapping for confidence interval:
            values = [np.random.choice(a, size=len(a),replace=True).mean() for i in range(500)]
            means[chan,coeff,ts] = np.array(values).mean()
            lowers[chan,coeff,ts], uppers[chan,coeff,ts] = np.percentile(values,[100*(1-confidence)/2,100*(1-(1-confidence)/2)])

        rvs1 = [A[i][chan, 1, ts] for i in range(len(subjects))]
        rvs2 = [A[i][chan, 2, ts] for i in range(len(subjects))]
        # Calculate t-test for coeff[1] vs coeff[2] (distance vs. direction):
        stat[chan,ts], p_val[chan, ts] = stats.ttest_ind(rvs1, rvs2)

print(f'Bootstrapping, took me {round(time.time() - start)} seconds...')

In [ ]:
# Save bootstrapping matrices:
np.save('regr_coeff_global_means.npy', means)    # .npy extension is added if not given
np.save('regr_coeff_global_lowers.npy', lowers)
np.save('regr_coeff_global_uppers.npy', uppers)

In [ ]:
# Save t-test matrices:
np.save('regr_coeff_stats.npy', stat)
np.save('regr_coeff_pval.npy', p_val)

In [ ]:
means = np.load('regr_coeff_global_means.npy')
lowers = np.load('regr_coeff_global_lowers.npy')
uppers = np.load('regr_coeff_global_uppers.npy')

In [ ]:
stat = np.load('regr_coeff_stats.npy')
p_val = np.load('regr_coeff_pval.npy')

In [ ]:
file_names = [f for f in listdir(paths[0]) if 'epo.fif' in f]

# Load file
file_name = file_names[0]
file = paths[0] + '/' + file_name
epochs = mne.read_epochs(file, preload=True)
names = epochs.ch_names

mapping = {'Fp1': (0,3), 'Fp2': (0,5), 'AF7': (1,0), 'AF3': (1,2), 'AF4': (1,6), 'AF8': (1,8),
           'F7': (2,0), 'F5': (2,1), 'F3': (2,2), 'F1': (2,3), 'Fz': (2,4), 'F2': (2,5), 'F4': (2,6), 'F6': (2,7), 'F8': (2,8),
           'FT7': (3,0), 'FC5': (3,1), 'FC3': (3,2), 'FC1': (3,3), 'FCz': (3,4), 'FC2': (3,5), 'FC4': (3,6), 'FC6': (3,7), 'FT8': (3,8),
           'T7': (4,0), 'C5': (4,1), 'C3': (4,2), 'C1': (4,3), 'Cz': (4,4), 'C2': (4,5), 'C4': (4,6), 'C6': (4,7), 'T8': (4,8),
           'TP7': (5,0), 'CP5': (5,1), 'CP3': (5,2), 'CP1': (5,3), 'CPz': (5,4), 'CP2': (5,5), 'CP4': (5,6), 'CP6': (5,7), 'TP8': (5,8),
           'P7': (6,0), 'P5': (6,1), 'P3': (6,2), 'P1': (6,3), 'Pz': (6,4), 'P2': (6,5), 'P4': (6,6), 'P6': (6,7), 'P8': (6,8),
           'PO7': (7,0), 'PO3': (7,2), 'POz': (7,4), 'PO4': (7,6), 'PO8': (7,8),
           'PO9': (8,1), 'O1': (8,3), 'Oz': (8,4), 'O2': (8,5), 'PO10': (8,7)
           }


In [ ]:
timepoints = np.arange(n_ts)
fig, axs = plt.subplots(9, 9)
# axs[0, 0].plot(x, y)+
fig.set_size_inches(18.5, 10.5)
fig.suptitle(f'Regression coefficients')
for chan in range(n_chan):
    x,y = mapping[names[chan]]
    for coeff in range(1,3):
        axs[x, y].plot(means[chan,coeff,:])
        axs[x, y].fill_between(timepoints, lowers[chan,coeff,:], uppers[chan,coeff,:], alpha=0.1)


plt.show()
fig.savefig('regr_topo_chans.svg')